---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: AI Engineering

### 📋 **Homework 4**: Embeddings & Semantic Search

### 📅 **Due Date**: Day of Lecture 5, 11:59 PM


**Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---

In this homework, you'll build on Homework 3 (BM25 search) by adding **embedding-based semantic search**.

You will:
1. **Generate embeddings** using both local (Hugging Face) and API (OpenAI) models
2. **Implement cosine similarity** from scratch
3. **Implement semantic search** from scratch
4. **Compare BM25 vs semantic search** using Recall
5. **Compare different embedding models** and analyze their differences

**Total Points: 95**

---

## Instructions

- Complete all tasks by filling in code where you see `# YOUR CODE HERE`
- You may use ChatGPT, Claude, documentation, Stack Overflow, etc.
- When using external resources, briefly cite them in a comment
- Run all cells before submitting to ensure they work

**Submission:**
1. Create a branch called `homework-4`
2. Commit and push your work
3. Create a PR and merge to main
4. Submit the `.ipynb` file on Blackboard

---

## Task 1: Environment Setup (10 points)

### 1a. Imports (5 pts)

Import the required libraries and load the WANDS data.

In [ ]:
# ruff: noqa: E402

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings("ignore")

# Import ONLY data loading from helpers
import sys
sys.path.append('../scripts')
from helpers import load_wands_products, load_wands_queries, load_wands_labels

# Embedding libraries - we use these directly
from sentence_transformers import SentenceTransformer
import litellm

# Load environment variables for API keys
from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_colwidth', 80)
print("All imports successful!")

In [ ]:
# Load the WANDS dataset
products = load_wands_products()
queries = load_wands_queries()
labels = load_wands_labels()

print(f"Products: {len(products):,}")
print(f"Queries: {len(queries):,}")
print(f"Labels: {len(labels):,}")

### 1b. Copy BM25 functions from HW3 (5 pts)

Copy your BM25 implementation from Homework 3. We'll use it to compare against semantic search.

In [ ]:
# Copy your BM25 functions from Homework 3
import Stemmer
import string
from collections import Counter

stemmer = Stemmer.Stemmer('english')
punct_trans = str.maketrans({key: ' ' for key in string.punctuation})

def snowball_tokenize(text: str) -> list[str]:
    """Tokenize text with Snowball stemming."""
    if pd.isna(text) or text is None:
        return []
    text = str(text).translate(punct_trans)
    tokens = text.lower().split()
    return [stemmer.stemWord(token) for token in tokens]

def build_index(docs: list[str], tokenizer) -> tuple[dict, list[int]]:
    """Build an inverted index from a list of documents."""
    index = {}
    doc_lengths = []
    for doc_id, doc in enumerate(docs):
        tokens = tokenizer(doc)
        doc_lengths.append(len(tokens))
        term_counts = Counter(tokens)
        for term, count in term_counts.items():
            if term not in index:
                index[term] = {}
            index[term][doc_id] = count
    return index, doc_lengths

def get_tf(term: str, doc_id: int, index: dict) -> int:
    """Get term frequency for a term in a document."""
    if term in index and doc_id in index[term]:
        return index[term][doc_id]
    return 0

def get_df(term: str, index: dict) -> int:
    """Get document frequency for a term."""
    if term in index:
        return len(index[term])
    return 0

def bm25_idf(df: int, num_docs: int) -> float:
    """BM25 IDF formula."""
    return np.log((num_docs - df + 0.5) / (df + 0.5) + 1)

def bm25_tf(tf: int, doc_len: int, avg_doc_len: float, k1: float = 1.2, b: float = 0.75) -> float:
    """BM25 TF normalization."""
    return (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * doc_len / avg_doc_len))

def score_bm25(query: str, index: dict, num_docs: int, doc_lengths: list[int],
               tokenizer, k1: float = 1.2, b: float = 0.75) -> np.ndarray:
    """Score all documents using BM25."""
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    avg_doc_len = np.mean(doc_lengths) if doc_lengths else 1.0
    for token in query_tokens:
        df = get_df(token, index)
        if df == 0:
            continue
        idf = bm25_idf(df, num_docs)
        if token in index:
            for doc_id, tf in index[token].items():
                tf_norm = bm25_tf(tf, doc_lengths[doc_id], avg_doc_len, k1, b)
                scores[doc_id] += idf * tf_norm
    return scores

def search_products(query: str, products_df: pd.DataFrame, index: dict,
                    doc_lengths: list[int], tokenizer, k: int = 10) -> pd.DataFrame:
    """Search products and return top-k results."""
    scores = score_bm25(query, index, len(products_df), doc_lengths, tokenizer)
    top_k_idx = np.argsort(-scores)[:k]
    results = products_df.iloc[top_k_idx].copy()
    results["score"] = scores[top_k_idx]
    results["rank"] = range(1, k + 1)
    return results

print('BM25 functions loaded from HW3!')


---

## Task 2: Understanding Embeddings (15 points)

### 2a. Load a local model and generate embeddings (5 pts)

Use `sentence-transformers` to load a local embedding model and generate embeddings for a list of words.

In [ ]:
# Load the all-MiniLM-L6-v2 model using SentenceTransformer
# Then generate embeddings for each word in the list
words = ["wooden coffee table", "oak dining table", "red leather sofa", "blue area rug", "kitchen sink"]

# YOUR CODE HERE
model_local = SentenceTransformer("all-MiniLM-L6-v2")
embeddings_local = model_local.encode(words)

# Print the number of embeddings you generated and the dimension of the embeddings
print(f"Generated {len(embeddings_local)} embeddings")
print(f"Embedding dimension: {embeddings_local.shape[1]}")
print(f"First embedding shape: {embeddings_local[0].shape}")


### 2b. Implement cosine similarity and create a similarity matrix (5 pts)

Implement cosine similarity from scratch:

$$\text{cosine\_similarity}(a, b) = \frac{a \cdot b}{\|a\| \times \|b\|}$$

In [ ]:
# Implement cosine similarity from scratch
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return dot_product / (norm_a * norm_b)

# Create similarity matrix
n = len(words)
sim_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        sim_matrix[i, j] = cosine_similarity(embeddings_local[i], embeddings_local[j])

# Display as DataFrame
pd.DataFrame(sim_matrix, index=words, columns=words)


### 2c. Embed using OpenAI API (5 pts)

Use `litellm` to get embeddings from OpenAI's API and compare dimensions.

In [ ]:
# Use litellm to get an embedding from OpenAI's text-embedding-3-small model
response = litellm.embedding(
    model="text-embedding-3-small",
    input=["wooden coffee table"]
)

embedding_openai = response.data[0]["embedding"]

# Compare the dimension with the local model
print(f"Local model dimension: {embeddings_local.shape[1]}")
print(f"OpenAI model dimension: {len(embedding_openai)}")

# Calculate ratio
ratio = len(embedding_openai) / embeddings_local.shape[1]
print(f"OpenAI embedding is {ratio:.1f}x larger than local embedding")


---

## Task 3: Batch Embedding Products (20 points)

### 3a. Embed a product sample (10 pts)

Create a combined text field and embed 5,000 products using the local model.

In [ ]:
# Get a consistent sample
products_sample = products.sample(5000, random_state=42).copy()
print(f"Sample size: {len(products_sample)}")


In [ ]:
# Create a combined text field (product_name + product_class)
# Then embed all products using model.encode()

# Combine fields - be extremely defensive against NaNs
products_sample["text"] = (
    products_sample["product_name"].fillna("").astype(str) + " " + 
    products_sample["product_class"].fillna("").astype(str)
)

# Double check: ensure every single item is a string
products_sample["text"] = products_sample["text"].astype(str)

# Generate embeddings (this may take a minute)
print("Generating embeddings... (this may take 1-2 minutes)")
product_embeddings = model_local.encode(products_sample["text"].tolist(), show_progress_bar=True)

print(f"Created embeddings with shape: {product_embeddings.shape}")


### 3b. Save and load embeddings (5 pts)

Save embeddings to a `.npy` file so you don't have to recompute them.

In [ ]:
# Save embeddings to temp/hw4_embeddings.npy
# Save products_sample to temp/hw4_products.csv
import os
os.makedirs("temp", exist_ok=True)

# Save
np.save("temp/hw4_embeddings.npy", product_embeddings)
products_sample.to_csv("temp/hw4_products.csv", index=False)

# Then load them back and verify they match
loaded_embeddings = np.load("temp/hw4_embeddings.npy")
loaded_products = pd.read_csv("temp/hw4_products.csv")

print(f"Embeddings shape: {loaded_embeddings.shape}")
print(f"Products shape: {loaded_products.shape}")

if np.allclose(product_embeddings, loaded_embeddings):
    print("Embeddings match perfectly!")
else:
    print("Embeddings do NOT match!")


### 3c. Cost estimation (5 pts)

Estimate the cost to embed all 43K products using OpenAI's API.

**Pricing**: text-embedding-3-small costs ~$0.02 per 1 million tokens.

In [ ]:
# Use tiktoken to count actual tokens in the sample
import tiktoken

try:
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
except:
    encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens for all texts in the sample
# Calculate tokens per product
texts = products_sample["text"].tolist()
total_tokens_sample = sum(len(encoding.encode(text)) for text in texts)
avg_tokens_per_product = total_tokens_sample / len(products_sample)

print(f"Total tokens in sample (5,000 products): {total_tokens_sample:,}")
print(f"Average tokens per product: {avg_tokens_per_product:.2f}")

# Then extrapolate to estimate cost for the full dataset (43K products)
total_products = len(products)
estimated_total_tokens = total_products * avg_tokens_per_product

cost_per_1m_tokens = 0.02  # $0.02 per 1M tokens
estimated_cost = (estimated_total_tokens / 1_000_000) * cost_per_1m_tokens

print(f"Estimated total tokens for {total_products:,} products: {estimated_total_tokens:,.0f}")
print(f"Estimated cost to embed full dataset: ${estimated_cost:.4f}")


---

## Task 4: Semantic Search (25 points)

### 4a. Implement semantic search (15 pts)

Implement a semantic search function from scratch.

In [ ]:
# Implement batch cosine similarity for efficiency
def batch_cosine_similarity(query_emb, doc_embs):
    """
    Calculate cosine similarity between a query embedding and many document embeddings.
    
    Args:
        query_emb: 1D array of shape (embedding_dim,)
        doc_embs: 2D array of shape (num_docs, embedding_dim)
        
    Returns:
        1D array of similarity scores
    """
    # Normalize query
    norm_query = np.linalg.norm(query_emb)
    if norm_query == 0:
        return np.zeros(doc_embs.shape[0])
    query_emb_norm = query_emb / norm_query
    
    # Normalize docs (can be pre-computed, but we do it here for simplicity)
    # Note: For strict cosine similarity, we divide dot product by product of norms
    # Equivalent to dot product of normalized vectors
    norm_docs = np.linalg.norm(doc_embs, axis=1)
    # Avoid division by zero
    norm_docs[norm_docs == 0] = 1e-10
    
    # Calculate dot product
    dot_products = np.dot(doc_embs, query_emb)
    
    # Divide by norms
    scores = dot_products / (norm_query * norm_docs)
    return scores


In [ ]:
# Implement semantic search
def semantic_search(query, model, doc_embeddings, products_df, k=10):
    """
    Search products using semantic embeddings.
    
    Args:
        query: Search query string
        model: SentenceTransformer model
        doc_embeddings: Numpy array of product embeddings
        products_df: DataFrame of products (must match order of embeddings)
        k: Number of results to return
        
    Returns:
        DataFrame with top-k products and similarity scores
    """
    # 1. Embed the query
    query_emb = model.encode(query)
    
    # 2. Calculate similarities
    scores = batch_cosine_similarity(query_emb, doc_embeddings)
    
    # 3. Get top-k results
    top_k_idx = np.argsort(-scores)[:k]
    
    # 4. Return results
    results = products_df.iloc[top_k_idx].copy()
    results["score"] = scores[top_k_idx]
    results["rank"] = range(1, k + 1)
    return results


In [ ]:
# Test semantic search
query = "modern coffee table"
results = semantic_search(query, model_local, product_embeddings, products_sample)

print(f"Query: {query}")
results[["product_name", "product_class", "score"]].head()


### 4b. Evaluate and compare BM25 vs semantic search (10 pts)

Implement Recall@k and compare the two search methods.

In [ ]:
def calculate_recall_at_k(relevant_ids, retrieved_ids, k=10):
    """
    Calculate Recall@k.
    
    Args:
        relevant_ids: Set of relevant product IDs (ground truth)
        retrieved_ids: List of retrieved product IDs (ordered)
        k: Cutoff rank
        
    Returns:
        Recall score (0.0 to 1.0)
    """
    if not relevant_ids:
        return 0.0
    
    # Get top-k retrieved
    top_k = set(retrieved_ids[:k])
    
    # Count intersection
    intersection = len(top_k.intersection(relevant_ids))
    
    # Calculate recall
    return intersection / len(relevant_ids)


In [ ]:
# Build BM25 index for the SAMPLE (to be fair)
index_sample, doc_lengths_sample = build_index(
    products_sample["product_name"].fillna("").tolist(), 
    snowball_tokenize
)

# Filter queries to those with relevant products IN OUR SAMPLE
# 1. Get all relevant product IDs for each query
relevant_map = labels[labels["grade"] > 0].groupby("query_id")["product_id"].apply(set).to_dict()

# 2. Identify products in our sample
sample_product_ids = set(products_sample["product_id"])

# 3. Filter queries: keep only if they have at least one relevant product in the sample
eval_queries = []
for _, row in queries.iterrows():
    qid = row["query_id"]
    if qid in relevant_map:
        # Intersection of relevant docs and our sample
        relevant_in_sample = relevant_map[qid].intersection(sample_product_ids)
        if relevant_in_sample:
            eval_queries.append({
                "query_id": qid,
                "query": row["query"],
                "relevant_ids": relevant_in_sample  # Only judge against what is possible to retrieve
            })

print(f"Evaluable queries (with relevant docs in sample): {len(eval_queries)}")


In [ ]:
# Evaluate both BM25 and semantic search on all filtered queries
bm25_recalls = []
semantic_recalls = []

print("Running evaluation...")
for q_item in eval_queries:
    query_text = q_item["query"]
    ground_truth = q_item["relevant_ids"]
    
    # 1. BM25 Search
    bm25_res = search_products(
        query_text, products_sample, index_sample, doc_lengths_sample, 
        snowball_tokenize, k=10
    )
    bm25_recall = calculate_recall_at_k(ground_truth, bm25_res["product_id"].tolist(), k=10)
    bm25_recalls.append(bm25_recall)
    
    # 2. Semantic Search
    # (We could check if we already embedded the query? No, just embed it)
    sem_res = semantic_search(query_text, model_local, product_embeddings, products_sample, k=10)
    sem_recall = calculate_recall_at_k(ground_truth, sem_res["product_id"].tolist(), k=10)
    semantic_recalls.append(sem_recall)

# Average Recall@10
avg_bm25_recall = np.mean(bm25_recalls)
avg_sem_recall = np.mean(semantic_recalls)

print(f"BM25 Average Recall@10: {avg_bm25_recall:.4f}")
print(f"Semantic Search Average Recall@10: {avg_sem_recall:.4f}")


In [ ]:
# Visualize comparison
methods = ["BM25", "Semantic Search"]
scores = [avg_bm25_recall, avg_sem_recall]

plt.figure(figsize=(8, 5))
plt.bar(methods, scores, color=["skyblue", "salmon"])
plt.ylabel("Recall@10")
plt.title("Search Performance Comparison (BM25 vs Semantic)")
plt.ylim(0, max(scores) * 1.2)
for i, v in enumerate(scores):
    plt.text(i, v + 0.01, f"{v:.4f}", ha="center")
plt.show()


---

## Task 5: Compare Embedding Models (20 points)

### 5a. Embed products with two different models (10 pts)

Compare embeddings from:
- `BAAI/bge-base-en-v1.5`
- `sentence-transformers/all-mpnet-base-v2`

In [ ]:
# Load the two embedding models
print("Loading BGE model...")
model_bge = SentenceTransformer("BAAI/bge-base-en-v1.5")

print("Loading MPNet model...")
model_mpnet = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

print("Models loaded!")


In [ ]:
# Embed products with both models
print("Embedding with BGE... (this may take 2-3 minutes)")
embeddings_bge = model_bge.encode(products_sample["text"].tolist(), show_progress_bar=True)

print("Embedding with MPNet... (this may take 2-3 minutes)")
embeddings_mpnet = model_mpnet.encode(products_sample["text"].tolist(), show_progress_bar=True)

print(f"BGE shape: {embeddings_bge.shape}")
print(f"MPNet shape: {embeddings_mpnet.shape}")


### 5b. Compare search results between models (10 pts)

Evaluate both models on the same queries and analyze differences.

In [ ]:
# Evaluate both models on all queries
bge_recalls = []
mpnet_recalls = []

print("Evaluating BGE and MPNet...")
for q_item in eval_queries:
    query_text = q_item["query"]
    ground_truth = q_item["relevant_ids"]
    
    # BGE Search
    # Note: BGE often requires a prompt like "Represent this sentence for searching relevant passages:"
    # But for simplicity/standard usage in sentence-transformers we often just encode
    # The model card says for queries: "Represent this sentence for searching relevant passages: "
    # Let's add the instruction for BGE queries as recommended
    bge_query_emb = model_bge.encode("Represent this sentence for searching relevant passages: " + query_text)
    bge_scores = batch_cosine_similarity(bge_query_emb, embeddings_bge)
    bge_top_k = np.argsort(-bge_scores)[:10]
    bge_recall = calculate_recall_at_k(ground_truth, products_sample.iloc[bge_top_k]["product_id"].tolist(), k=10)
    bge_recalls.append(bge_recall)
    
    # MPNet Search
    mpnet_query_emb = model_mpnet.encode(query_text)
    mpnet_scores = batch_cosine_similarity(mpnet_query_emb, embeddings_mpnet)
    mpnet_top_k = np.argsort(-mpnet_scores)[:10]
    mpnet_recall = calculate_recall_at_k(ground_truth, products_sample.iloc[mpnet_top_k]["product_id"].tolist(), k=10)
    mpnet_recalls.append(mpnet_recall)

print(f"BGE Average Recall@10: {np.mean(bge_recalls):.4f}")
print(f"MPNet Average Recall@10: {np.mean(mpnet_recalls):.4f}")


In [ ]:
# Compare results for specific queries
test_queries = ["comfortable sofa", "star wars rug", "modern coffee table"]

for query in test_queries:
    print(f"\n--- Query: {query} ---")
    
    # BGE
    bge_q_emb = model_bge.encode("Represent this sentence for searching relevant passages: " + query)
    bge_scores = batch_cosine_similarity(bge_q_emb, embeddings_bge)
    bge_top_1 = products_sample.iloc[np.argmax(bge_scores)]["product_name"]
    
    # MPNet
    mpnet_q_emb = model_mpnet.encode(query)
    mpnet_scores = batch_cosine_similarity(mpnet_q_emb, embeddings_mpnet)
    mpnet_top_1 = products_sample.iloc[np.argmax(mpnet_scores)]["product_name"]
    
    print(f"BGE Top Result:   {bge_top_1}")
    print(f"MPNet Top Result: {mpnet_top_1}")


In [ ]:
# Visualize model comparison with a scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(bge_recalls, mpnet_recalls, alpha=0.5)
plt.xlabel("BGE Recall@10")
plt.ylabel("MPNet Recall@10")
plt.title("Per-Query Recall Comparison")
plt.plot([0, 1], [0, 1], "r--")  # Diagonal line
plt.grid(True, alpha=0.3)
plt.show()


---

## Task 6: Git Submission (5 points)

Submit your work using the Git workflow:

- [ ] Create a new branch called `homework-4`
- [ ] Commit your work with a meaningful message
- [ ] Push to GitHub
- [ ] Create a Pull Request
- [ ] Merge the PR to main
- [ ] Submit the `.ipynb` file on Blackboard

The TA will verify your submission by checking the merged PR on GitHub.